# Calculate gain matrix restricted to occipital lobe

- create surface source spaces from __all__ vertices in white matter
- find vertices that match the union of `{l,r}h.RM.annot`-vertices, use
    - `pos['rr'] ~ src[hemi_idx]['rr']`: locations restricted to labels
    - `pos['nn'] ~ src[hemi_idx]['nn']`: surface normals restricted to labels
    - `pos['verts']`: indices into original surface for each `rr`
- use `setup_volume_source_space` with the `pos`-dict to create a discrete source space
- create a standard BEM
- calculate the fwd-operator using the BEM, save to disk

In [1]:
import numpy as np
from os.path import join
from mne import setup_source_space, setup_volume_source_space, SourceSpaces
from mne import read_labels_from_annot
from mne import (make_bem_model, make_bem_solution,
                 make_forward_solution, write_forward_solution)
from collections import OrderedDict
from functools import reduce
from operator import add

In [2]:
data_path = '/Users/cjb/projects/CFF_Retinotopy/scratch'
subject = '030_WAH'
subjects_dir = join(data_path, 'fs_subjects_dir')
fname_raw = join(data_path, 'VS1_cropped1sec.fif')
fname_trans = join(data_path, '030_WAH-trans.fif')
fname_fwd = join(data_path, 'restricted_1LBEM-fwd.fif')

First load the surface source spaces

In [3]:
orig_src = setup_source_space('030_WAH', fname=None, spacing='all',
                              surface='white', add_dist=False,
                              subjects_dir=subjects_dir)

Setting up the source space with the following parameters:

SUBJECTS_DIR = /Users/cjb/projects/CFF_Retinotopy/scratch/fs_subjects_dir
Subject      = 030_WAH
Surface      = white
Include all vertices

>>> 1. Creating the source space...

Loading /Users/cjb/projects/CFF_Retinotopy/scratch/fs_subjects_dir/030_WAH/surf/lh.white...
    Triangle neighbors and vertex normals...
loaded lh.white 162628/162628 selected to source space (all)

Loading /Users/cjb/projects/CFF_Retinotopy/scratch/fs_subjects_dir/030_WAH/surf/rh.white...
    Triangle neighbors and vertex normals...
loaded rh.white 158658/158658 selected to source space (all)

You are now one step closer to computing the gain matrix


In [6]:
all_labels = OrderedDict()
use_verts = dict()
use_tris = dict()
for hemi_idx, hemi in enumerate(['lh', 'rh']):
    labels = read_labels_from_annot(subject,
                                    parc='RM', regexp='RM',
                                    subjects_dir=subjects_dir,
                                    hemi=hemi)
    comb_label = reduce(add, labels)
    use_verts[hemi] = np.where(np.in1d(orig_src[hemi_idx]['vertno'],
                                       comb_label.vertices))[0]
    # find all triangles with at least one included vertex
    rows = np.any(np.in1d(orig_src[hemi_idx]['tris'].ravel(),
                          use_verts[hemi].ravel()).\
                  reshape(orig_src[hemi_idx]['tris'].shape), axis=1)
    use_tris[hemi] = orig_src[hemi_idx]['tris'][rows]

Reading labels from parcellation..
   read 21 labels from /Users/cjb/projects/CFF_Retinotopy/scratch/fs_subjects_dir/030_WAH/label/lh.RM.annot
[done]
Reading labels from parcellation..
   read 21 labels from /Users/cjb/projects/CFF_Retinotopy/scratch/fs_subjects_dir/030_WAH/label/rh.RM.annot
[done]


In [7]:
for hidx, hemi in enumerate(['lh', 'rh']):
    orig_src[hidx]['vertno'] = use_verts[hemi]
    orig_src[hidx]['nuse'] = len(use_verts[hemi])
    orig_src[hidx]['use_tris'] = use_tris[hemi]
    orig_src[hidx]['nuse_tri'] = len(use_tris[hemi])
    orig_src[hidx]['inuse'] = np.zeros(orig_src[hidx]['np'], int)
    orig_src[hidx]['inuse'][use_verts[hemi]] = 1

## Forward calculation using BEM

### Uncomment these for diagnostic images

In [9]:
# %matplotlib qt

In [10]:
# from mne.viz import plot_bem
# plot_bem(subject=subject, subjects_dir=subjects_dir,
#          brain_surfaces='white', src=orig_src, orientation='coronal')

In [11]:
# from mayavi import mlab  # noqa
# from surfer import Brain  # noqa

# brain = Brain(subject, 'lh', 'inflated', subjects_dir=subjects_dir)
# surf = brain._geo

# vertidx = np.where(orig_src[0]['inuse'])[0]
# # vertidx = pos['vertno'][0]

# mlab.points3d(surf.x[vertidx], surf.y[vertidx],
#               surf.z[vertidx], color=(1, 1, 0), scale_factor=1.0)

In [12]:
conductivity = (0.3,)  # for single layer
# conductivity = (0.3, 0.006, 0.3)  # for three layers
model = make_bem_model(subject=subject, ico=4,
                       conductivity=conductivity,
                       subjects_dir=subjects_dir)
bem = make_bem_solution(model)

Creating the BEM geometry...
Going from 5th to 4th subdivision of an icosahedron (n_tri: 20480 -> 5120)
inner skull CM is   1.23 -24.49  25.31 mm
Surfaces passed the basic topology checks.
Complete.

Approximation method : Linear collocation

Homogeneous model surface loaded.
Computing the linear collocation solution...
    Matrix coefficients...
        inner_skull (2562) -> inner_skull (2562) ...
    Inverting the coefficient matrix...
Solution ready.
BEM geometry computations complete.


In [13]:
fwd = make_forward_solution(fname_raw, trans=fname_trans, src=orig_src,
                            bem=bem, fname=None, meg=True, eeg=False,
                            mindist=5.0, n_jobs=2)

Source space                 : <SourceSpaces: [<surface (lh), n_vertices=162628, n_used=4661, coordinate_frame=MRI (surface RAS)>, <surface (rh), n_vertices=158658, n_used=5039, coordinate_frame=MRI (surface RAS)>]>
MRI -> head transform source : /Users/cjb/projects/CFF_Retinotopy/scratch/030_WAH-trans.fif
Measurement data             : VS1_cropped1sec.fif
BEM model                    : dict
Accurate field computations
Do computations in head coordinates
Free source orientations
Destination for the solution : None

Read 2 source spaces a total of 9700 active source locations

Coordinate transformation: MRI (surface RAS) -> head
     0.999768  0.010701  0.018671      -1.60 mm
    -0.014853  0.970947  0.238832      23.19 mm
    -0.015572 -0.239054  0.970881      25.68 mm
     0.000000  0.000000  0.000000       1.00

Read 306 MEG channels from info
72 coil definitions read
Coordinate transformation: MEG device -> head
     0.998327 -0.055338  0.016745       5.77 mm
     0.053596  0.994415

[Parallel(n_jobs=2)]: Done   3 out of   2 | elapsed:    4.0s remaining:   -1.3s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    4.0s finished


Thank you for waiting.

Setting up compensation data...
    No compensation set. Nothing more to do.

Composing the field computation matrix...


[Parallel(n_jobs=2)]: Done   3 out of   2 | elapsed:    4.8s remaining:   -1.6s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    4.8s finished


Computing MEG at 9700 source locations (free orientations)...


[Parallel(n_jobs=2)]: Done   3 out of   2 | elapsed:    3.3s remaining:   -1.1s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    3.3s finished
[Parallel(n_jobs=2)]: Done   3 out of   2 | elapsed:   10.2s remaining:   -3.4s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:   10.2s finished
[Parallel(n_jobs=2)]: Done   3 out of   2 | elapsed:    3.9s remaining:   -1.3s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    3.9s finished



Finished.


In [14]:
write_forward_solution(fname_fwd, fwd, overwrite=True)

Overwriting existing file.
    Write a source space...
    [done]
    Write a source space...
    [done]
    2 source spaces written
